# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up Graphlab Create

In [1]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1530851555.log


This non-commercial license of GraphLab Create for academic use is assigned to dlihit@ufl.edu and will expire on May 22, 2019.


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [36]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 0.485661     | 6869667.895833     | 385998.458623 |

| 6         | 7        | 0.000002  | 0.521680     | 6847177.773672     | 380824.455891 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [38]:
model_all.coefficients.print_rows(18, 4)

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None |  274873.05595 |  None  |
|     bedrooms     |  None |      0.0      |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 8468.53108691 |  None  |
|   sqft_living    |  None | 24.4207209824 |  None  |
| sqft_living_sqrt |  None | 350.060553386 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 842.068034898 |  None  |
|    sqft_above    |  None | 20.0247224171 |  None  |
|  sqft_basement   |  None |

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [16]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [17]:
def getRSS(model, testData):
    predictions = model.predict(testData)
    rssVector = (predictions - testData['price'])**2
    return sum(rssVector)

In [18]:
import numpy as np
for l1 in np.logspace(1, 7, num=13):
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1, verbose=False)
    rss = getRSS(model, validation)
    print "l1: " + str(l1) + ", rss: " + str(rss)

l1: 10.0, rss: 6.25766285142e+14
l1: 31.6227766017, rss: 6.25766285362e+14
l1: 100.0, rss: 6.25766286058e+14
l1: 316.227766017, rss: 6.25766288257e+14
l1: 1000.0, rss: 6.25766295212e+14
l1: 3162.27766017, rss: 6.25766317206e+14
l1: 10000.0, rss: 6.25766386761e+14
l1: 31622.7766017, rss: 6.25766606749e+14
l1: 100000.0, rss: 6.25767302792e+14
l1: 316227.766017, rss: 6.25769507644e+14
l1: 1000000.0, rss: 6.25776517727e+14
l1: 3162277.66017, rss: 6.25799062845e+14
l1: 10000000.0, rss: 6.25883719085e+14


*** QUIZ QUESTION. *** What was the best value for the `l1_penalty`?

In [ ]:
l1: 10.0, rss: 6.25766285142e+14

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [39]:
model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=10.0, verbose=False)
model.coefficients.print_rows(18, 4)

+------------------+-------+------------------+--------+
|       name       | index |      value       | stderr |
+------------------+-------+------------------+--------+
|   (intercept)    |  None |  18993.4272128   |  None  |
|     bedrooms     |  None |  7936.96767903   |  None  |
| bedrooms_square  |  None |  936.993368193   |  None  |
|    bathrooms     |  None |  25409.5889341   |  None  |
|   sqft_living    |  None |  39.1151363797   |  None  |
| sqft_living_sqrt |  None |  1124.65021281   |  None  |
|     sqft_lot     |  None | 0.00348361822299 |  None  |
|  sqft_lot_sqrt   |  None |  148.258391011   |  None  |
|      floors      |  None |   21204.335467   |  None  |
|  floors_square   |  None |  12915.5243361   |  None  |
|    waterfront    |  None |  601905.594545   |  None  |
|       view       |  None |  93312.8573119   |  None  |
|    condition     |  None |  6609.03571245   |  None  |
|      grade       |  None |  6206.93999188   |  None  |
|    sqft_above    |  None |  4

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [21]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [22]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [24]:
for l1 in l1_penalty_values:
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1, verbose=False)
    rss = getRSS(model, validation)
    nnz = model['coefficients']['value'].nnz()
    print "l1: " + str(l1) + ", rss: " + str(rss) + ", nnz: " + str(nnz)

l1: 100000000.0, rss: 6.27492659875e+14, nnz: 18
l1: 127427498.57, rss: 6.28210516771e+14, nnz: 18
l1: 162377673.919, rss: 6.29176689541e+14, nnz: 18
l1: 206913808.111, rss: 6.30650082719e+14, nnz: 18
l1: 263665089.873, rss: 6.32940229287e+14, nnz: 17
l1: 335981828.628, rss: 6.3626814023e+14, nnz: 17
l1: 428133239.872, rss: 6.41261198311e+14, nnz: 17
l1: 545559478.117, rss: 6.48983455376e+14, nnz: 17
l1: 695192796.178, rss: 6.60962217696e+14, nnz: 17
l1: 885866790.41, rss: 6.77261520728e+14, nnz: 16
l1: 1128837891.68, rss: 7.01046815867e+14, nnz: 15
l1: 1438449888.29, rss: 7.37850622829e+14, nnz: 15
l1: 1832980710.83, rss: 7.9616310964e+14, nnz: 13
l1: 2335721469.09, rss: 8.69018172894e+14, nnz: 12
l1: 2976351441.63, rss: 9.66925692362e+14, nnz: 10
l1: 3792690190.73, rss: 1.08186759232e+15, nnz: 6
l1: 4832930238.57, rss: 1.24492736032e+15, nnz: 5
l1: 6158482110.66, rss: 1.38416149024e+15, nnz: 3
l1: 7847599703.51, rss: 1.23079472046e+15, nnz: 1
l1: 10000000000.0, rss: 1.22915716064e+15

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [34]:
nnz = 10
l1 = 3399000000
while nnz >= 7:
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1, verbose=False)
    rss = getRSS(model, validation)
    nnz = model['coefficients']['value'].nnz()
    if(nnz == 7):
        print "l1: " + str(l1) + ", rss: " + str(rss)
    if(nnz < 7):
        break
    l1 = l1 + 1000000

l1_penalty_min = 0
l1_penalty_max = 0

l1: 3414000000, rss: 1.04411288988e+15
l1: 3415000000, rss: 1.0441929941e+15
l1: 3416000000, rss: 1.04427310719e+15
l1: 3417000000, rss: 1.04435322914e+15
l1: 3418000000, rss: 1.04443335996e+15
l1: 3419000000, rss: 1.04451352017e+15
l1: 3420000000, rss: 1.04459366872e+15
l1: 3421000000, rss: 1.04467382615e+15
l1: 3422000000, rss: 1.04475401296e+15
l1: 3423000000, rss: 1.04483418812e+15
l1: 3424000000, rss: 1.04491437215e+15
l1: 3425000000, rss: 1.04499456505e+15
l1: 3426000000, rss: 1.04507476681e+15
l1: 3427000000, rss: 1.04515499797e+15
l1: 3428000000, rss: 1.04523521747e+15
l1: 3429000000, rss: 1.04531544584e+15
l1: 3430000000, rss: 1.04539570361e+15
l1: 3431000000, rss: 1.04547594971e+15
l1: 3432000000, rss: 1.04555620468e+15
l1: 3433000000, rss: 1.04563646852e+15
l1: 3434000000, rss: 1.04571674122e+15
l1: 3435000000, rss: 1.04579704335e+15
l1: 3436000000, rss: 1.04587733379e+15
l1: 3437000000, rss: 1.04595763309e+15
l1: 3438000000, rss: 1.04603796183e+15
l1: 3439000000, rss: 1.046

In [41]:
l1 = 3414000000
model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1, verbose=False)
rss = getRSS(model, validation)
nnz = model['coefficients']['value'].nnz()
print "l1: " + str(l1) + ", rss: " + str(rss) + ", nnz: " + str(nnz)
model.coefficients.print_rows(18, 4)

l1: 3414000000, rss: 1.04411288988e+15, nnz: 7
+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None | 220362.381399 |  None  |
|     bedrooms     |  None | 786.731241997 |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 16049.4585874 |  None  |
|   sqft_living    |  None | 32.5610497478 |  None  |
| sqft_living_sqrt |  None | 698.522259956 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None |  2955.8268728 |  None  |
|    sqft_above    |  None | 30.238

***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [ ]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?